In [1]:
LOAD_MODEL = 'models/seq2seq_2.01.pt'
MODEL_CHECKPOINT = 'models/seq2seq_2.01.pt'
DATASET_PATH = 'data/interim/preprocessed_paranmt.tsv'

In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader

import os
os.chdir("..") # go to the root dir

# Get the Dataset

In [3]:
MAX_SENT_SIZE = 10
MAX_TOKENS = 8_000

In [4]:
from src.data.make_dataset import ParanmtDataset

train_dataset = ParanmtDataset(
    path=DATASET_PATH,
    max_sent_size=MAX_SENT_SIZE,
    train=True,
    seed=42,
)

In [5]:
train_dataset.build_vocab(
    min_freq=2,
    specials=['<unk>', '<pad>', '<sos>', '<eos>'],
    max_tokens=MAX_TOKENS,
)

In [6]:
enc_vocab = train_dataset.toxic_vocab
dec_vocab = train_dataset.neutral_vocab

print("size of encoder vocab:", len(enc_vocab))
print("size of decoder vocab:", len(dec_vocab))

size of encoder vocab: 8000
size of decoder vocab: 8000


In [7]:
val_dataset = ParanmtDataset(
    path=DATASET_PATH,
    max_sent_size=MAX_SENT_SIZE,
    vocabs=(enc_vocab, dec_vocab), # avoid data leakage
    train=False,
    seed=42,
    take_first=10_000,
)

In [8]:
train_dataset.df

,similarity,lenght_diff,toxic_sent,neutral_sent,toxic_val,neutral_val
0,0.708038,0.171429,"[what, the, hell, i, danger, looking, at, ?]","[what, the, hell, is, safe, watch, ?]",0.888703,0.130954
1,0.606822,0.238095,"[lisa, ,, hit, him, again, .]","[lisa, ,, one, more, .]",0.957538,0.000053
2,0.719271,0.051282,"[what, are, you, doing, with, that, hooker, ?]","[what, are, you, doing, with, that, outsider, ?]",0.998877,0.000056
3,0.821008,0.047619,"[we, are, going, to, hit, him, !]","[it, is, going, to, hit, !]",0.997299,0.014387
4,0.725030,0.096774,"[i, do, not, fucking, believe, it, !]","[i, do, not, freaking, believe, it]",0.957814,0.056393
...,...,...,...,...,...,...
157735,0.827812,0.200000,"[I, will, make, you, fall, !]","[I, am, going, to, fall, !]",0.590488,0.006672
157736,0.625040,0.333333,"[i, fucking, my, girlfriend, .]","[satisfying, my, girl, behind, my, back, .]",0.999578,0.029578
157737,0.815115,0.041667,"[he, is, going, to, shoot, again, .]","[he, is, going, to, fire, again, .]",0.989201,0.008294
157738,0.866068,0.037037,"[oh, ,, mars, solid, ,, you, stink, .]","[oh, ,, mars, solid, ,, you, smell, .]",0.999077,0.072257


In [9]:
len(train_dataset), len(val_dataset)

(157740, 10000)

## Build the Dataloaders

In [10]:
batch_size = 128

In [11]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
)

In [12]:
# let's check if shape and everything is ok
for batch in train_dataloader:
    toxic_sent, neutral_sent = batch
    print("toxic_sent.shape:", toxic_sent.shape)
    print("neutral_sent.shape:", neutral_sent.shape)
    break

toxic_sent.shape: torch.Size([128, 10])
neutral_sent.shape: torch.Size([128, 10])


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Load the Model

- Simple EncoderDecoder (Seq2Seq) architerture
- a little trick was used (every time the decoder carries with it context vector from the encoder)

In [14]:
from src.models.seq2seq.encoder import Encoder
from src.models.seq2seq.decoder2 import Decoder2 # NOTE: using different Decoder than first notebook
from src.models.seq2seq import Seq2Seq

In [15]:
INPUT_DIM = len(enc_vocab)
OUTPUT_DIM = len(dec_vocab)
EMBED_DIM = 256
NUM_HIDDEN = 512
N_LAYERS = 1
DROPOUT = 0.5
ENC_PADDING_IDX = enc_vocab['<pad>']
DEC_PADDING_IDX = dec_vocab['<pad>']

In [16]:
# load the encoder and decoder for our model
encoder = Encoder(
    input_dim=INPUT_DIM,
    embed_dim=EMBED_DIM,
    hidden_dim=NUM_HIDDEN,
    num_layers=N_LAYERS,
    dropout=DROPOUT,
    vocab=enc_vocab,
    padding_idx=ENC_PADDING_IDX
).to(device)

decoder = Decoder2(
    output_dim=OUTPUT_DIM,
    embed_dim=EMBED_DIM,
    hidden_dim=NUM_HIDDEN,
    num_layers=N_LAYERS,
    dropout=DROPOUT,
    vocab=dec_vocab,
    padding_idx=DEC_PADDING_IDX
).to(device)

In [17]:
best_loss = float('inf')

model = Seq2Seq(
    encoder=encoder,
    decoder=decoder,
    device=device,
    max_sent_size=MAX_SENT_SIZE,
).to(device)

In [18]:
from src.models.utils import count_parameters

print(f"number of parameters in model: {count_parameters(model)//1e6}M")

number of parameters in model: 18.0M


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(ignore_index=dec_vocab['<pad>'])

In [20]:
from src.models.train_model import train

best_loss = train(
    model=model,
    loaders=(train_dataloader, val_dataloader),
    optimizer=optimizer,
    criterion=criterion,
    epochs=20,
    device=device,
    best_loss=best_loss,
    ckpt_path=MODEL_CHECKPOINT,
    clip_grad=1,
    teacher_force={
        'value': 0.8,
        'gamma': 1.0,
        'update_every_n_epoch': 50,
    } # first 5 epoch teacher force 1, after it will be turned off
)

Evaluating 20: 100%|██████████| 79/79 [00:00<00:00, 88.54it/s, loss=3.6]  


In [21]:
# let's load the model and predict
model = torch.load(MODEL_CHECKPOINT)
model.to(device)
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (vocab): Vocab()
    (embedding): Embedding(8000, 256, padding_idx=1)
    (rnn): LSTM(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder2(
    (embedding): Embedding(8000, 256, padding_idx=1)
    (rnn): LSTM(768, 512, batch_first=True)
    (fc_out): Linear(in_features=1280, out_features=8000, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (vocab): Vocab()
  )
)

In [22]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()

# let's see how our model works
num_examples = 10
num_sentence = 3
dataset = train_dataset
for _ in range(num_examples):
    idx = np.random.randint(0, len(dataset))
    toxic_sent = detokenizer.detokenize(dataset.df.loc[idx, 'toxic_sent'])
    neutral_sent = detokenizer.detokenize(dataset.df.loc[idx, 'neutral_sent'])
    
    print('toxic_sent:', toxic_sent)
    print('neutral_sent:', neutral_sent)
    # let's use beam search
    preds = model.predict(
        toxic_sent,
        beam=True,
        beam_search_num_candidates=num_sentence,
        post_process_text=False,
    )
    print("predictions:")
    for i in range(num_sentence):
        print(f"\t{i+1})", preds[i])
    print("\n")

toxic_sent: you heard from that nigga, keith?
neutral_sent: did you hear from keith?
predictions:
	1) ['did', 'you', 'hear', 'that', 'keith', 'keith', 'keith', '?', '<eos>']
	2) ['did', 'you', 'hear', 'that', 'keith', 'keith', '?', '<eos>']
	3) ['did', 'you', 'hear', 'that', 'keith', ',', 'keith', '?', '<eos>']


toxic_sent: do you understand this fucker?
neutral_sent: do you maggots understand?
predictions:
	1) ['do', 'you', 'maggots', 'understand', '?', '<eos>']
	2) ['do', 'you', 'understand', 'this', '?', '<eos>']
	3) ['do', 'you', 'understand', 'that', '?', '<eos>']


toxic_sent: are you crazy?
neutral_sent: dinner! are you mad?
predictions:
	1) ['are', 'you', 'mad', '?', '<eos>']
	2) ['have', 'you', 'lost', 'your', 'mind', '?', '<eos>']
	3) ['have', 'you', 'gone', 'mad', '?', '<eos>']


toxic_sent: ellen will destroy you.
neutral_sent: ellen, you will be crushed.
predictions:
	1) ['ellen', 'will', 'destroy', 'you', '.', '<eos>']
	2) ['ellen', 'ellen', 'will', 'destroy', 'you', '.'